## Exploring the Sem Eval 2018 dataset using Autolabel

#### Set up the API keys for the providers you want to use

In [ ]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxx'

#### Install the autolabel library

In [ ]:
!pip install 'refuel-autolabel[openai]'

#### Download the dataset

In [1]:
from autolabel import get_data

get_data('twitter_emotion_detection')

2023-06-28 10:07:51 autolabel.utils ERROR: twitter_emotion_detection not in list of available datasets: ['banking', 'civil_comments', 'ledgar', 'walmart_amazon', 'company', 'squad_v2', 'sciq', 'conll2003', 'movie_reviews']. Exiting...


This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

In [2]:
import json

from autolabel import LabelingAgent

In [3]:
# load the config
with open('config_twitter_emotion_detection.json', 'r') as f:
    config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `multilabel_classification` (since it's a multi label classification task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.labels`: `['neutral', 'anger', 'anticipation', 'disgust', 'fear', ...]` (the full list of labels to choose from)
* `prompt.task_guidelines`: `'You are an expert at classifying tweets as...` (how we describe the task to the LLM)
* `prompt.few_shot_num`: 5 (how many labeled examples to provide to the LLM)

In [4]:
config

{'task_name': 'EmotionClassification',
 'task_type': 'multilabel_classification',
 'dataset': {'label_column': 'labels', 'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'You are an expert at classifying tweets as neutral or one or more of the given emotions that best represent the mental state of the poster.\nYour job is to correctly label the provided input example into one or more of the following categories:\n{labels}',
  'output_guidelines': 'You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"',
  'labels': ['neutral',
   'anger',
   'anticipation',
   'disgust',
   'fear',
   'joy',
   'love',
   'optimism',
   'pessimism',
   'sadness',
   'surprise',
   'trust'],
  'few_shot_examples': 'seed.csv',
  'few_shot_selection': 'semantic_similarity',
  'few_shot_num': 5,
  'example_template': 'Input: {example}\nOutput: {labels}'}}

In [5]:
# create an aggent for labeling
agent = LabelingAgent(config=config)

In [6]:
agent.plan('test.csv')

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $4.9705 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0025 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at classifying tweets as neutral or one or more of the given emotions that best represent the mental state of the poster.
Your job is to correctly label the provided input example into one or more of the following categories:
neutral
anger
anticipation
disgust
fear
joy
love
optimism
pessimism
sadness
surprise
trust

You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"

Some examples with their output answers are provided below:

Input: @MaryamNSharif I think just becz u have so much terror in pak nd urself being  a leader u forgot d difference btw a leader nd terrorist !
Output: anger, disgust, fear

Input: In wake of fresh #terror threat and sounding of alert in #Mumbai, praying for safety &amp; security of everybody in the city #Maharashtra #news
Output: fear

Input: Somewhere I rd a rpt tht Pakis wr afraid of TSD &amp; asked it 2 shut dn. Congis obliged &amp; exposed it,hounded them.time 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [7]:
# now, do the actual labeling
labels, df, metrics_list = agent.run('test.csv', max_items=100)

Output()

2023-06-28 10:12:01 openai INFO: error_code=None error_message='The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID f36689e030a5793f1557f2ead8f49ab6 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-06-28 10:12:01 langchain.embeddings.openai WARNING: Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID f36689e030a5793f1557f2ead8f49ab6 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or co

Actual Cost: 0.0029


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.4889 │ 100     │ 0.08     │ 1.0             │
└────────┴─────────┴──────────┴─────────────────┘

Total number of failures: 0

We are at a 0.4889 f1 when labeling the first 100 examples. Let's see if we can use confidence scores to improve f1 further by removing the less confident examples from our labeled set.

### Compute confidence scores

In [8]:
# Start computing confidence scores (using Refuel's LLMs)
# os.environ['REFUEL_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxx'

In [9]:
# set `compute_confidence` -> True
config["model"]["compute_confidence"] = True

In [10]:
agent = LabelingAgent(config=config)

In [11]:
agent.plan('test.csv')

Output()

2023-06-28 10:13:54 openai INFO: error_code=None error_message='The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 48d623f577c2b89cc2ba681a363097ec in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-06-28 10:13:54 langchain.embeddings.openai WARNING: Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 48d623f577c2b89cc2ba681a363097ec in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or co

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $4.9705 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0025 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at classifying tweets as neutral or one or more of the given emotions that best represent the mental state of the poster.
Your job is to correctly label the provided input example into one or more of the following categories:
neutral
anger
anticipation
disgust
fear
joy
love
optimism
pessimism
sadness
surprise
trust

You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"

Some examples with their output answers are provided below:

Input: @MaryamNSharif I think just becz u have so much terror in pak nd urself being  a leader u forgot d difference btw a leader nd terrorist !
Output: anger, disgust, fear

Input: In wake of fresh #terror threat and sounding of alert in #Mumbai, praying for safety &amp; security of everybody in the city #Maharashtra #news
Output: fear

Input: Somewhere I rd a rpt tht Pakis wr afraid of TSD &amp; asked it 2 shut dn. Congis obliged &amp; exposed it,hounded them.time 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [12]:
labels, df, metrics_list = agent.run('test.csv', max_items=100)

Output()

2023-06-28 10:16:44 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


2023-06-28 10:17:31 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


2023-06-28 10:17:47 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


2023-06-28 10:17:50 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


...

2023-06-28 10:18:31 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


2023-06-28 10:18:34 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


Metric: auroc: 0.7663
Actual Cost: 0.0025


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ f1     ┃ support ┃ accuracy ┃ completion_rate ┃ threshold ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ 0.6667 │ 1       │ 0.0      │ 0.01            │ 0.9555    │
│ 0.8333 │ 2       │ 0.5      │ 0.02            │ 0.9248    │
│ 0.7143 │ 7       │ 0.1429   │ 0.07            │ 0.8177    │
│ 0.75   │ 8       │ 0.25     │ 0.08            │ 0.8099    │
│ 0.7467 │ 10      │ 0.2      │ 0.1             │ 0.7784    │
│ 0.7697 │ 11      │ 0.2727   │ 0.11            │ 0.7746    │
│ 0.7611 │ 12      │ 0.25     │ 0.12            │ 0.765     │
│ 0.7795 │ 13      │ 0.3077   │ 0.13            │ 0.7606    │
│ 0.65   │ 28      │ 0.1429   │ 0.28            │ 0.6927    │
│ 0.6621 │ 29      │ 0.1724   │ 0.29            │ 0.6805    │
│ 0.6625 │ 34      │ 0.1471   │ 0.34            │ 0.6213    │
│ 0.6721 │ 35      │ 0.1714   │ 0.35            │ 0.6114    │
│ 0.6198 │ 48      │ 0.125    │ 0.48            │ 0.5561    │
│ 0.6276 │ 49      │ 0.1429   │ 0.49            │ 0.5553    │
│ 0.5745 │ 60      │ 0.1167   │ 0.6             │ 0.5011    │
│ 0.5815 │ 61      │ 0.1311   │ 0.61            │ 0.4988    │
│ 0.4889 │ 100     │ 0.08     │ 1.0             │ 0.0206    │
└────────┴─────────┴──────────┴─────────────────┴───────────┘

Total number of failures: 0

Looking at the table above, we can see that if we set the confidence threshold at `0.6114`, we are able to label at 0.67 f1 and getting a completion rate of 35%. This means, we would ignore all the data points where confidence score is less than `0.6114` (which would end up being around 65% of all samples). This would, however, guarantee a higher quality labeled dataset for us.